# Portfolio Optimization within a Bayesian Framework
> Optimal portfolio weights accounting for aleatoric and epistemic uncertainty.

- toc: true 
- badges: true
- comments: true
- categories: [probabilistic programming, bayesian, portfolio optimization]
- image: https://github.com/jpwoeltjen/researchBlog/blob/master/images/bayesian_portfolio_opt.png





The Bayesian framework for portfolio optimization offers very important advantages compared to the classical approach. Among them are the following:

- The Bayesian framework accounts for full uncertainty:

  - Aleatoric uncertainty: inherent variance of returns.
  - Epistemic uncertainty: parameter uncertainty due to limited data (at every step).

- The specification of complex models with constraints is natural and estimation via MCMC straightforward.



# The optimization problem

The general idea is to (numerically) maximize the expected utility based on a large number of sample draws from the posterior predictive distribution of returns:

\begin{equation}
\begin{aligned}
\max _{\boldsymbol{\omega}} \mathrm{E}\left[U\left(\boldsymbol{\omega}^{\prime} \mathbf{r}_{T+1}\right)\right]=& \max _{\boldsymbol{\omega}} \int U\left(\boldsymbol{\omega}^{\prime} \mathbf{r}_{T+1}\right) p\left(\mathbf{r}_{T+1} \mid \mathbf{\mathcal{D}}\right) \mathrm{d} \mathbf{r}_{T+1} \\
\text { subject to } & \boldsymbol{\omega}^{\prime} \mathbf{1}=1,
\end{aligned}
\end{equation}

where:
- $\mathbf{r}_{T+1}$ :  vector of returns at $T+1$.
- $\mathbf{\mathcal{D}}$ : predictive and observable features.
- $U(\cdot)$ : utility function (quadratic utility is unrealistic; utility function should punish large losses but not large gains).
- $\boldsymbol{\omega}$ : vector of portfolio weights. 
- $\mathbf{\Sigma}$ : covariance matrix of returns. 
- $\lambda$ : risk aversion parameter.

If the utility function is quadratic:
\begin{equation}
\mathrm{E}[U(\boldsymbol{\omega})]=\boldsymbol{\omega}^{\prime} \mathrm{E}(\mathbf{r})-\lambda \boldsymbol{\omega}^{\prime} \mathbf{\Sigma} \boldsymbol{\omega}.
\end{equation}



# Implementation

To model high-dimensional returns with covariance, coskewness, and cokurtosis it is convenient (and also plausible) to assume a factor model.
For each $t, t=1, \ldots, T,$ estimate the cross-sectional regression  and obtain the time series of factor return estimates $F,$ and specific return estimates $\widehat{E}$
$$
\widehat{\mathbf{F}}=\left(\begin{array}{llll}
\widehat{f}_{1,1} & \widehat{f}_{1,2} & \cdots & \widehat{f}_{1, K} \\
\cdots & \cdots & \cdots & \cdots \\
\widehat{f}_{t, 1} & \widehat{f}_{t, 2} & \cdots & \widehat{f}_{t, K} \\
\cdots & \cdots & \cdots & \hat{f}_{T, K}
\end{array}\right)
$$
and
$$
\widehat{\mathbf{E}}=\left(\begin{array}{llll}
\widehat{e}_{1,1} & \widehat{e}_{1,2} & \cdots & \widehat{e}_{1, N} \\
\cdots & \cdots & \cdots & \cdots \\
\widehat{e}_{t, 1} & \widehat{e}_{t, 2} & \cdots & \widehat{e}_{t, N} \\
\cdots & \cdots & \cdots & \widehat{e}_{T, N}
\end{array}\right)
$$
respectively.



Predict (simulate) the expected returns, factor returns, and stock-specific returns and use them as drivers to compute scenarios of the stock returns according to
$$
\tilde{\mathbf{r}}=\tilde{\boldsymbol{\mu}} +{\mathbf{B} \tilde{\mathbf{f}}}+\tilde{\mathbf{e}}
$$
where $\tilde{\boldsymbol{\mu}}$ is the matrix of simulated mean returns for $N$ assets, generated from the posterior predictive distribution of the conditional expectation model, and $\tilde{\mathbf{f}}$ and $\tilde{\mathbf{e}}$ are the predicted (simulated) factor and stock-specific returns, respectively.

- Use high-frequency (HF) data to extact factors (including industries).
- Use HF data to compute loadings.
- Use Bayesian (realized) MV-GARCH to forecast factor returns.
- Use Bayesian (realized) GARCH to forecast stock specific returns (if all factors are accounted for these can be assumed independent)
- Estimate the posterior predictive distribution of the conditional expectation $\boldsymbol{\mu}$ (for example via a Bayesian neural network) and sample from it.
- Simulate 10,000 MC draws according to $
\tilde{\mathbf{r}}=\tilde{\boldsymbol{\mu}} +{\mathbf{B} \tilde{\mathbf{f}}}+\tilde{\mathbf{e}}
$.

- Optimize (empirical; based on the sample draws) expected utility.

There are GARCH-type models that account for coskewness (GRJ-GARCH) and cokurtosis (BEKK). GARCH based on Student's t innovations are better able to capture heaviness of tails. 
Minimizing the expected shortfall subject to an expected return constraint is a valid portfolio optimization procedure in a non-Gaussian setting (where the standard deviation is not an appropriate risk measure). 